In [ ]:
%pylab inline
import pandas as pd
#import psycopg2
import sklearn
import seaborn as sns
import matplotlib
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, 
AdaBoostClassifier)
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sqlalchemy import create_engine
sns.set_style("white")

In [ ]:
db_name = "appliedda"
hostname = "10.10.2.10"
schema = 'M3'

In [ ]:
pgsql_engine = create_engine( "postgresql://10.10.2.10/appliedda" )

In [ ]:
full_data = pd.read_sql("SELECT * FROM M3.prisonerwages", con = pgsql_engine) 

In [ ]:
#print out all the columns for easy copying later
for i in full_data.columns:
   print "'"+i+"',"

In [ ]:
pd.set_option('display.max_columns', 300)

In [ ]:
#pd.options.display.max_seq_items = 20000

In [ ]:
#from notebook.services.config import ConfigManager
#cm = ConfigManager().update('notebook', {'limit output': 100000})

In [ ]:
#f = open("fileofoutput.txt", "w")

In [ ]:
#print >>f , full_data.isnull().sum()

In [ ]:
#f.close()

In [ ]:
#Create birth decade variable
full_data['birthdecade']=(full_data['birth_year']-(full_data['birth_year']%10)).astype('object')
#full_data[['birthdecade','birth_year']].head(25)

In [ ]:
#Dummy variable for any employment in 8 quarters
full_data['employed'] = full_data.filter(regex = "^wageq[0-8]$").max(axis = 1).apply(lambda x: (x > 0))
full_data['employed'].value_counts(normalize=True)

In [ ]:
#Dummy variable for any employment in q0
full_data['employed0'] = full_data.filter(regex = "wageq0").max(axis = 1).apply(lambda x: (x > 0))
full_data['employed0'].value_counts(normalize=True)

In [ ]:
#Dummy variable for any employment in q1
full_data['employed1'] = full_data.filter(regex = "wageq1").max(axis = 1).apply(lambda x: (x > 0))
full_data['employed1'].value_counts(normalize=True)

In [ ]:
#Dummy variable for any employment in q2
full_data['employed2'] = full_data.filter(regex = "wageq2").max(axis = 1).apply(lambda x: (x > 0))
full_data['employed2'].value_counts(normalize=True)

In [ ]:
#Dummy variable for any employment in q3
full_data['employed3'] = full_data.filter(regex = "wageq3").max(axis = 1).apply(lambda x: (x > 0))
full_data['employed3'].value_counts(normalize=True)

In [ ]:
#Dummy variable for any employment in q4
full_data['employed4'] = full_data.filter(regex = "wageq4").max(axis = 1).apply(lambda x: (x > 0))
full_data['employed4'].value_counts(normalize=True)

In [ ]:
#Dummy variable for any employment in q5
full_data['employed5'] = full_data.filter(regex = "wageq5").max(axis = 1).apply(lambda x: (x > 0))
full_data['employed5'].value_counts(normalize=True)

In [ ]:
#Dummy variable for any employment in q6
full_data['employed6'] = full_data.filter(regex = "wageq6").max(axis = 1).apply(lambda x: (x > 0))
full_data['employed6'].value_counts(normalize=True)

In [ ]:
#Dummy variable for any employment in q7
full_data['employed7'] = full_data.filter(regex = "wageq7").max(axis = 1).apply(lambda x: (x > 0))
full_data['employed7'].value_counts(normalize=True)

In [ ]:
#Number of quarters employed
li = ['employed0', 'employed1', 'employed2', 'employed3', 'employed4', 'employed5', 'employed6', 'employed7']
full_data['numemplqtr'] = full_data[li].sum(axis=1)

In [ ]:
full_data['numemplqtr'].value_counts(ascending=True, normalize=True)

In [ ]:
full_data[['wageq0', 'wageq1', 'wageq2','wageq3', 'wageq4', 'wageq5', 'wageq6', 'wageq7', 'employed', 'numemplqtr']].describe(percentiles=[.45, .5, .55])

In [ ]:
full_data.head(n=50)

In [ ]:
#First employment quarter
def begemp(row):
    if row['employed0'] == 1:
        return 0
    elif row['employed1'] == 1:
        return 1
    elif row['employed2'] == 1:
        return 2
    elif row['employed3'] == 1:
        return 3
    elif row['employed4'] == 1:
        return 4
    elif row['employed5'] == 1:
        return 5
    elif row['employed6'] == 1:
        return 6
    elif row['employed7'] == 1:
        return 7
    else:
        return np.nan 


In [ ]:
full_data['firstemp'] = full_data.apply(lambda row: begemp (row), axis = 1)

In [ ]:
full_data['firstemp'].value_counts(ascending=True, dropna=False)

In [ ]:
full_data['firstemp'].value_counts(ascending=True, normalize= True, dropna=False)

In [ ]:
#create variable indicating whether 8 quarter salary is below poverty line for 1 person
full_data['below_poverty_on_avg']=1-(full_data.filter(regex = "^wageq[0-9]$").sum(axis = 1)-full_data.filter(regex = "^baselinepovertyq[0-9]$").sum(axis = 1)>0)

In [ ]:
full_data['below_poverty_on_avg'].value_counts(ascending=True, dropna=False)

In [ ]:
#Dummy variables for below poverty line in each quarter
full_data['belpov0']=1-(full_data.filter(regex = "wageq0").sum(axis = 1)-full_data.filter(regex = "baselinepovertyq0").sum(axis = 1)>0)
full_data['belpov1']=1-(full_data.filter(regex = "wageq1").sum(axis = 1)-full_data.filter(regex = "baselinepovertyq1").sum(axis = 1)>0)
full_data['belpov2']=1-(full_data.filter(regex = "wageq2").sum(axis = 1)-full_data.filter(regex = "baselinepovertyq2").sum(axis = 1)>0)
full_data['belpov3']=1-(full_data.filter(regex = "wageq3").sum(axis = 1)-full_data.filter(regex = "baselinepovertyq3").sum(axis = 1)>0)
full_data['belpov4']=1-(full_data.filter(regex = "wageq4").sum(axis = 1)-full_data.filter(regex = "baselinepovertyq4").sum(axis = 1)>0)
full_data['belpov5']=1-(full_data.filter(regex = "wageq5").sum(axis = 1)-full_data.filter(regex = "baselinepovertyq5").sum(axis = 1)>0)
full_data['belpov6']=1-(full_data.filter(regex = "wageq6").sum(axis = 1)-full_data.filter(regex = "baselinepovertyq6").sum(axis = 1)>0)
full_data['belpov7']=1-(full_data.filter(regex = "wageq7").sum(axis = 1)-full_data.filter(regex = "baselinepovertyq7").sum(axis = 1)>0)

#full_data['belpov0'].value_counts(normalize=True)

In [ ]:
full_data[['belpov0','belpov1','belpov2','belpov3','belpov4','belpov5','belpov6','belpov7']].head(n=10)

In [ ]:
full_data[['wageq0', 'belpov0', 'baselinepovertyq0']].head(n=25)

In [ ]:
#Number of quarters below poverty
li = ['belpov0', 'belpov1', 'belpov2', 'belpov3', 'belpov4', 'belpov5', 'belpov6', 'belpov7']
full_data['numbelpovqtr'] = full_data[li].sum(axis=1)

In [ ]:
full_data['numbelpovqtr'].value_counts(ascending=True, normalize=True)

In [ ]:
#create concatenated string of employment pattern
for i in range(0, 8):
    full_data['q'+str(i)+'s'] = (full_data['employed'+str(i)] - 0).astype('object')

In [ ]:
#check results of a quarter
full_data['q2s'].value_counts(ascending=True, dropna=False).astype(str)

In [ ]:
full_data['emp_pattern'] = full_data['q0s'].astype(str) + full_data['q1s'].astype(str) + full_data['q2s'].astype(str) + full_data['q3s'].astype(str) + full_data['q4s'].astype(str) + full_data['q5s'].astype(str) + full_data['q6s'].astype(str) + full_data['q7s'].astype(str)

In [ ]:
full_data['emp_pattern'].value_counts(ascending=True, normalize=False)

In [ ]:
#create concatenated string of below pov pattern
for i in range(0, 8):
    full_data['pq'+str(i)+'s'] = (full_data['belpov'+str(i)] - 0).astype('object')

In [ ]:
#check results of a quarter
full_data['pq2s'].value_counts(ascending=True, dropna=False).astype(str)

In [ ]:
full_data['pov_pattern'] = full_data['pq0s'].astype(str) + full_data['pq1s'].astype(str) + full_data['pq2s'].astype(str) + full_data['pq3s'].astype(str) + full_data['pq4s'].astype(str) + full_data['pq5s'].astype(str) + full_data['pq6s'].astype(str) + full_data['pq7s'].astype(str)

In [ ]:
full_data['pov_pattern'].value_counts(ascending=False, normalize=False)

In [ ]:
#Above poverty quarter
def begabvpov(row):
    if row['belpov0'] == 0:
        return 0
    elif row['belpov1'] == 0:
        return 1
    elif row['belpov2'] == 0:
        return 2
    elif row['belpov3'] == 0:
        return 3
    elif row['belpov4'] == 0:
        return 4
    elif row['belpov5'] == 0:
        return 5
    elif row['belpov6'] == 0:
        return 6
    elif row['belpov7'] == 0:
        return 7
    else:
        return np.nan 

In [ ]:
full_data['firstabvpov'] = full_data.apply(lambda row: begabvpov (row), axis = 1)

In [ ]:
full_data['firstabvpov'].value_counts(ascending=True, dropna=False)

In [ ]:
full_data['firstabvpov'].value_counts(ascending=True, normalize=True, dropna=False)

In [ ]:
# create time served variable, rounded to nearest year
full_data['prisontime']=full_data['prisontime'].round().astype('object')

In [ ]:
#create flag for whether employed in quarter prior to admission
full_data['anypriorwage']=(full_data['priorwage']>0)-0

In [ ]:
#earned time binary vars
full_data['meritorious_good_time']=(full_data['gttyp15']>0)-0
full_data['education_in_prison']=(full_data['gttyp17']>0)-0
full_data['substanceabuse_treatment']=(full_data['gttyp18']>0)-0
full_data['working_in_prison']=(full_data['gttyp19']>0)-0
good_time_vars=['meritorious_good_time','education_in_prison','substanceabuse_treatment','working_in_prison']

for i in good_time_vars:
    print(i)
    print(full_data[i].value_counts(normalize=True))

In [ ]:
full_data[['birthdecade',
'drugalcf',
'drugcocf',
'drugampf',
'drugmarf',
'drugherf',
'drugpcpf',
'drugothf',
'drugunkf',
'sexreg',
'sexoff',
'gangact',
'marstat',
'exittyp',
'hclass',
'lstsclvl',
'admtypo',
'sex',
'race',
'exityr',
'kids',
'curadmyr',
'prisontime', 
'anypriorwage', 
'meritorious_good_time',
'education_in_prison',
'substanceabuse_treatment',
'working_in_prison']].dtypes

In [ ]:
descriptives=['birthdecade',
'drugalcf',
'drugcocf',
'drugampf',
'drugmarf',
'drugherf',
'drugpcpf',
'drugothf',
'drugunkf',
'sexreg',
'sexoff',
'gangact',
'marstat',
'exittyp',
'hclass',
'lstsclvl',
'admtypo',
'sex',
'race',
'exityr',
'kids',
'curadmyr',
'prisontime', 
'anypriorwage', 
'meritorious_good_time',
'education_in_prison',
'substanceabuse_treatment',
'working_in_prison']
for i in descriptives:
    print(i)
    print(full_data[i].groupby(full_data['employed']).value_counts(normalize=False))

In [ ]:
descriptives=['birthdecade',
'drugalcf',
'drugcocf',
'drugampf',
'drugmarf',
'drugherf',
'drugpcpf',
'drugothf',
'drugunkf',
'sexreg',
'sexoff',
'gangact',
'marstat',
'exittyp',
'hclass',
'lstsclvl',
'admtypo',
'sex',
'race',
'exityr',
'kids',
'curadmyr',
'prisontime', 
'anypriorwage', 
'meritorious_good_time',
'education_in_prison',
'substanceabuse_treatment',
'working_in_prison']
for i in descriptives:
    print(i)
    print(full_data[i].groupby(full_data['below_poverty_on_avg']).value_counts(normalize=False))

In [ ]:
DrugVars=['drugalcf','drugampf','drugcocf','drugherf','drugmarf','drugothf','drugpcpf','drugunkf']

In [ ]:
for i in DrugVars:
    full_data[i]=full_data[i].replace(['X','F',NaN],['Y','Y','N'])
    print(i)
    print(full_data[i].value_counts(normalize=True))

In [ ]:
full_data.to_sql("prisonerwagesdesc2", con = pgsql_engine, schema = 'm3')

In [ ]:
##STOP

In [ ]:
###START HERE
varlist = ['drugalcf','drugampf']
for i in varlist:
    desdf = pd.value_counts(full_data.i).reset_index()
    desdf.columns = ['varvalue', 'frequency']
    desdf['variable'] = i
    print(desdf)

In [ ]:
full_data.groupby('drugalcf')['employed'].count()

In [ ]:
DrugVars=['drugalcf','drugampf','drugcocf','drugherf','drugmarf','drugothf','drugpcpf','drugunkf']
for i in DrugVars:
    full_data[i]=full_data[i].replace(['X','F',NaN],['Y','Y','N'])
    print(i)
    print(full_data[i].value_counts(normalize=True))

In [ ]:
pd.crosstab(full_data['drugalcf'], full_data['drugcocf'])

In [ ]:
#subset of those who are employed
employ_data = full_data.loc[full_data['numemplqtr'] > 0]
employ_data.head()

In [ ]:
employ_data['numemplqtr'].value_counts(ascending=True, normalize=True)

In [ ]:
employ_data[['wageq0', 'wageq1', 'wageq2','wageq3', 'wageq4', 'wageq5', 'wageq6', 'wageq7', 'employed', 'numemplqtr']].describe(percentiles=[.45, .5, .55])

In [ ]:
#create a substet of variables to check employed variable
chkqtr = full_data[['wageq0', 'wageq1', 'wageq2','wageq3', 'wageq4', 'wageq5', 'wageq6', 'wageq7', 'employed', 'numemplqtr', 'employed0', 'employed1', 'employed2', 'employed3', 'employed4', 'employed5', 'employed6', 'employed7']]
chkqtr.head()

In [ ]:
#generate random smaple of n observations to check employed variable
randchkqtr = chkqtr.sample(n = 20)
randchkqtr.head(n = 20)

In [ ]:
randchkqtr.describe(percentiles=[.45, .5, .55])

In [ ]:
#create copy with all NaNs = 0 to get descriptives that include people with no wages
full_data0 = full_data
full_data0.head()

In [ ]:
full_data0.fillna(0, inplace=True)

In [ ]:
full_data0[['wageq0', 'wageq1', 'wageq2','wageq3', 'wageq4', 'wageq5', 'wageq6', 'wageq7']].describe(percentiles=[.45, .5, .55])

In [ ]:
full_data['kids'].value_counts(normalize=False, sort=True)

In [ ]:
full_data[['kids']].isnull().sum()

In [ ]:
sub_data = full_data[['docnbr',
'birthyr',
'race',
'sex',
'curadmdt',
'curadmmo',
'curadmyr',
'admtypo',
'relinst',
'lstsclvl',
'hclass',
'hofnscd',
'exittyp',
'actmsrdt',
'actmsrmo',
'actmsryr',
'actdisdt',
'actdismo',
'actdisyr',
'cccadmdt',
'cccadmmo',
'cccadmyr',
'jailtime',
'marstat',
'kids',
'educlvl',
'gang',
'vetf',
'drugalcf',
'drugcocf',
'drugampf',
'drugmarf',
'drugherf',
'drugpcpf',
'drugothf',
'drugunkf',
'empplanf',
'gangact',
'birthpl',
'citzshp',
'zipcode',
'sexreg',
'sexoff',
'exitda',
'exitmo',
'exityr',
'birth_year',
'birth_date',
'curadm_date',
'actdis_date',
'exit_date',
'zip5',
                     ]]

In [ ]:
sub_data['employed'] = full_data.filter(regex = "^wageq[0-8]$").max(axis = 1).apply(lambda x: (x > 0))
sub_data['employed'].value_counts(normalize=True)

In [ ]:
full_data[['wageq0', 'wageq1', 'wageq2','wageq3', 'wageq4', 'wageq5', 'wageq6', 'wageq7', 'numemplqtr']].describe(percentiles=[.45, .5, .55])

In [ ]:
emp_data = full_data.loc[full_data['numemplqtr'] > 0]

In [ ]:
emp_data.head()

In [ ]:
emp_data[['wageq0', 'wageq1', 'wageq2','wageq3', 'wageq4', 'wageq5', 'wageq6', 'wageq7', 'numemplqtr']].describe(percentiles=[.45, .5, .55])

In [ ]:
f = open("fileofoutput.txt", "w")

##integers

print >>f ,sub_data[[
'birthyr',   
'curadmdt',
'curadmmo', 
'curadmyr', 
'hofnscd', 
'actmsrdt',  
'actmsrmo', 
'actmsryr', 
'actdisdt', 
'actdismo', 
'actdisyr', 
'cccadmdt', 
'cccadmmo', 
'cccadmyr', 
'jailtime', 
'kids', 
'exitda',     
'exitmo',   
'exityr', 
'employed'
]].describe()

sub_data[[
'birthyr',   
'curadmdt',
'curadmmo', 
'curadmyr', 
'hofnscd', 
'actmsrdt',  
'actmsrmo', 
'actmsryr', 
'actdisdt', 
'actdismo', 
'actdisyr', 
'cccadmdt', 
'cccadmmo', 
'cccadmyr', 
'jailtime', 
'kids', 
'exitda',     
'exitmo',   
'exityr', 
'birthyr',
'employed'
]].describe()

In [ ]:
f = open("fileofoutput.txt", "a")

##integers

print >>f ,sub_data[[
'birthyr',
'employed'
]].describe()

sub_data[[
'birthyr',
'employed'
]].describe()

In [ ]:
f = open("fileofoutput.txt", "a")

print >>f ,full_data[['cccadmdt',
'cccadmmo',
'cccadmyr',
'jailtime',
'marstat',
'kids',
'educlvl',
'gang',
'vetf',
'drugalcf',
'drugcocf',
'drugampf',
'drugmarf',
'drugherf',
'drugpcpf',
'drugothf',
'drugunkf',
'empplanf',
'gangact',
]].describe()

full_data[['cccadmdt','cccadmmo','cccadmyr','jailtime','marstat','kids','educlvl','gang','vetf','drugalcf','drugcocf','drugampf','drugmarf','drugherf','drugpcpf','drugothf','drugunkf','empplanf','gangact',
]].describe()

In [ ]:
f = open("fileofoutput.txt", "a")

print >>f ,full_data[['birthpl',
'citzshp',
'zipcode',
'sexreg',
'sexoff',
'exitda',
'exitmo',
'exityr',
'birth_year',
'birth_date',
'curadm_date',
'actdis_date',
'exit_date',
'zip5'
]].describe()                      

In [ ]:
f.close()

In [ ]:
sub_data.dtypes

In [ ]:
sub_data.employed = sub_data.employed.astype(float)

In [ ]:
sub_data['drugalcf'].value_counts(normalize=False, sort=True)

In [ ]:
#SELECT DISTINCT(curadmyr), COUNT(*)
#FROM m3.wagebyqtr_forcohort
#GROUP BY curadmyr;